In [160]:
import math

import numpy as np

In [161]:
def point(x, y):
    return np.array([x, y])

In [162]:
def rotate_point_clockwise(point, deg_rotation):
    rad_rotation = math.radians(deg_rotation)
    rotation_matrix = np.array(
        [
            [math.cos(rad_rotation), math.sin(rad_rotation)],
            [-math.sin(rad_rotation), math.cos(rad_rotation)],
        ]
    )
    return np.dot(rotation_matrix, point)


# Rotate around the centroid of the polygon (average of all points)
def rotate_polygon_clockwise(points, deg_rotation):
    total_x = 0
    total_y = 0
    for point in points:
        total_x = total_x + point[0]
        total_y = total_y + point[0]
    center = np.array([total_x, total_y]) / len(points)
    # Now subtract the center point to get the shape centered about origin
    centered_points = [point - center for point in points]
    centered_points = [
        rotate_point_clockwise(point, deg_rotation) for point in centered_points
    ]
    return [point + center for point in centered_points]

In [164]:
def ballistic_sim(initial_velocity, elevation, target_height, delta_t):
    t = 0
    x = 0
    y = 0
    v_0 = math.sin(math.radians(elevation)) * initial_velocity
    v_y = math.sin(math.radians(elevation)) * initial_velocity
    v_x = math.cos(math.radians(elevation)) * initial_velocity
    while v_y > 0 or y > target_height:
        t = t + delta_t
        x = x + v_x * delta_t
        y = y + v_y * delta_t
        v_y = v_0 - 9.81 * t
    return x, t


def ballistic_sim2(initial_velocity, elevation, target_height, delta_t):
    f = 0.00031  # from arma for 81mm
    t = 0
    x = 0
    y = 0
    v_y = math.sin(math.radians(elevation)) * initial_velocity
    v_x = math.cos(math.radians(elevation)) * initial_velocity
    while v_y > 0 or y > target_height:
        v = math.sqrt(v_x**2 + v_y**2)
        deceleration = f * v**2
        v_theta = math.atan(v_y / v_x)
        # decel_x = -1 * math.cos(v_theta) * deceleration
        # decel_y = -1 * math.sin(v_theta) * deceleration
        decel_x = -1 * deceleration
        decel_y = -1 * deceleration
        decel_y = decel_y - 9.81

        t = t + delta_t
        x = x + v_x * delta_t
        y = y + v_y * delta_t
        v_x = v_x + (decel_x * delta_t)
        v_y = v_y + (decel_y * delta_t)
    return x, t


def ballistic_sim3(initial_velocity, elevation, target_height, delta_t, include_drag):
    """Calculates the distance and time of flight for a projectile launched
    with `initial_velocity` m/s, `elevation` degrees, landing at `target_height`
    relative height from launch height, with simulation timestep of `delta_t`
    """
    plot_points = []
    # 81mm Initial Velocity - 250m/s * 1.5
    # 105mm Initial Velocity - 850m/s * .25
    f = 0.00031  # 81mm - .00031, 105mm - .0043
    m = 0.25  # 81mm - .25, 105mm - 23
    t = 0
    position = np.array([0, 0])
    gravity = np.array([0, -9.81])
    velocity = initial_velocity * np.array(
        [math.cos(math.radians(elevation)), math.sin(math.radians(elevation))]
    )
    while velocity[1] > 0 or position[1] > target_height:
        plot_points.append(position)

        speed = np.linalg.norm(velocity)
        drag_speed = f * speed * speed / m
        drag_direction = -1 * (velocity / speed)
        drag_vector = drag_speed * drag_direction
        deceleration = drag_vector + gravity if include_drag else gravity

        velocity = velocity + deceleration * delta_t
        position = position + velocity * delta_t
        t = t + delta_t
    return position[0], t, plot_points


# FIXME: Not taking into account relative height from observer, which means I'm
# calculating eastings/northings based on hypotenuse of the triangle rather
# than the actual x/y distance
def azimuth_from_observer_relative(
    mortar_easting: float,
    mortar_northing: float,
    observer_easting: float,
    observer_northing: float,
    obs_to_enemy_azimuth: float,
    obs_to_enemy_distance: float,
):
    # First need to calculate easting and northing of enemy
    # relative to observer
    obs_to_enemy_easting = 0.0
    obs_to_enemy_northing = 0.0

    # Due North
    if obs_to_enemy_azimuth == 0:
        obs_to_enemy_easting = 0
        obs_to_enemy_northing = obs_to_enemy_distance
    # First Quadrant
    elif obs_to_enemy_azimuth < 90:
        angle = 90 - obs_to_enemy_azimuth
        obs_to_enemy_easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_northing = math.sin(math.radians(angle)) * obs_to_enemy_distance
    # Due East
    elif obs_to_enemy_azimuth == 90:
        obs_to_enemy_easting = obs_to_enemy_distance
        obs_to_enemy_northing = 0
    # Second Quadrant
    elif obs_to_enemy_azimuth < 180:
        angle = obs_to_enemy_azimuth - 90
        obs_to_enemy_easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due South
    elif obs_to_enemy_azimuth == 180:
        obs_to_enemy_easting = 0
        obs_to_enemy_northing = -1 * obs_to_enemy_distance

    # Third Quadrant
    elif obs_to_enemy_azimuth < 270:
        angle = 270 - obs_to_enemy_azimuth
        obs_to_enemy_easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due West
    elif obs_to_enemy_azimuth == 270:
        obs_to_enemy_easting = -1 * obs_to_enemy_distance
        obs_to_enemy_northing = 0
    # Fourth Quadrant
    else:
        angle = obs_to_enemy_azimuth - 270
        obs_to_enemy_easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_northing = math.sin(math.radians(angle)) * obs_to_enemy_distance

    enemy_easting = observer_easting + obs_to_enemy_easting
    enemy_northing = observer_northing + obs_to_enemy_northing

    distance = distancefunc(
        mortar_easting, mortar_northing, enemy_easting, enemy_northing
    )

    return (
        azimuth_from_grids(
            mortar_easting, mortar_northing, enemy_easting, enemy_northing
        ),
        distance,
    )


def distancefunc(x1, y1, x2, y2):
    return math.sqrt(math.pow((x2 - x1), 2) + math.pow((y2 - y1), 2))


def azimuth_from_grids(start_easting, start_northing, end_easting, end_northing):
    east_vector = end_easting - start_easting
    north_vector = end_northing - start_northing

    # since we want azimuth, we need to shift the axes
    # east becomes north, and north becomes west

    temp = north_vector
    north_vector = east_vector
    east_vector = temp

    initial = math.degrees(math.atan2(north_vector, east_vector))
    if initial < 0:
        initial = 360 + initial

    return initial


def enemy_pos_from_observer_relative(
    mortar_easting,
    mortar_northing,
    observer_easting,
    observer_northing,
    obs_to_enemy_azimuth,
    obs_to_enemy_distance,
):
    # First need to calculate easting and northing of enemy
    # relative to observer
    obs_to_enemy_easting = 0
    obs_to_enemy_northing = 0

    # Due North
    if obs_to_enemy_azimuth == 0:
        obs_to_enemy_easting = 0
        obs_to_enemy_northing = obs_to_enemy_distance
    # First Quadrant
    elif obs_to_enemy_azimuth < 90:
        angle = 90 - obs_to_enemy_azimuth
        obs_to_enemy_easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_northing = math.sin(math.radians(angle)) * obs_to_enemy_distance
    # Due East
    elif obs_to_enemy_azimuth == 90:
        obs_to_enemy_easting = obs_to_enemy_distance
        obs_to_enemy_northing = 0
    # Second Quadrant
    elif obs_to_enemy_azimuth < 180:
        angle = obs_to_enemy_azimuth - 90
        obs_to_enemy_easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due South
    elif obs_to_enemy_azimuth == 180:
        obs_to_enemy_easting = 0
        obs_to_enemy_northing = -1 * obs_to_enemy_distance

    # Third Quadrant
    elif obs_to_enemy_azimuth < 270:
        angle = 270 - obs_to_enemy_azimuth
        obs_to_enemy_easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due West
    elif obs_to_enemy_azimuth == 270:
        obs_to_enemy_easting = -1 * obs_to_enemy_distance
        obs_to_enemy_northing = 0
    # Fourth Quadrant
    else:
        angle = obs_to_enemy_azimuth - 270
        obs_to_enemy_easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_northing = math.sin(math.radians(angle)) * obs_to_enemy_distance

    enemy_easting = observer_easting + obs_to_enemy_easting
    enemy_northing = observer_northing + obs_to_enemy_northing
    return enemy_easting, enemy_northing


def calculate_elevation(
    initial_velocity: float, target_height: float, distance: float, upper_half: bool
):
    # Check if target is even reachable using optimal launch angle
    current_elevation = 45
    current_distance = ballistic_sim3(
        initial_velocity, current_elevation, target_height, 0.001, True
    )[0]
    if current_distance < distance:
        # target unreachable
        return HTMLResponse("Target Unreachable.", status_code=400)

    if upper_half:
        max_elevation = 90
        min_elevation = 45
    else:
        max_elevation = 45
        min_elevation = 0
    while abs(current_distance - distance) > 1:
        current_elevation = (min_elevation + max_elevation) / 2
        current_distance = ballistic_sim3(
            initial_velocity, current_elevation, target_height, 0.001, True
        )[0]
        if upper_half:
            if current_distance < distance:
                max_elevation = current_elevation
            else:
                min_elevation = current_elevation
    return {
        "elevation": round(current_elevation, 1),
        "time_to_impact": round(
            ballistic_sim3(
                initial_velocity, current_elevation, target_height, 0.001, True
            )[1],
            1,
        ),
    }


def elevation_regression(distance):
    if distance > 1500:
        return -175
    else:
        return (
            603
            + -0.347 * distance
            + -1.28e-04 * distance**2
            + 1.64e-07 * distance**3
            + -1.03e-10 * distance**4
        )

In [165]:
def generate_ballistics_table(initial_velocity, air_drag, heights=None):
    if heights == None:
        heights = [height for height in range(0, -10000, -100)]
    else:
        heights = [-1 * height for height in heights]
    for height in heights:
        result = ballistic_sim3(initial_velocity, 0, height, 0.001, False)
        print("x: ", result[0], "y: -", height, "z: ", result[1])

In [166]:
reforger_tools_drops = [
    16.672001,
    37.368,
    65.996002,
    102.334999,
    123.328003,
    146.169006,
    170.830002,
    197.285004,
    225.509995,
    255.477997,
    287.165985,
    320.548004,
    355.600006,
    392.298004,
    430.618011,
    470.536987,
    512.031006,
    555.078979,
    599.656006,
    645.742004,
    693.315002,
    742.348999,
    792.828003,
    844.728027,
    898.026001,
    952.70697,
    1008.742981,
    1066.119019,
    1124.812988,
    1184.807983,
    1246.081055,
    1277.192993,
    1308.61499,
    1340.348022,
    1372.38501,
    1404.733032,
    1437.383057,
    1470.332031,
    1503.577026,
    1537.119995,
    1570.958008,
    1605.088013,
    1639.505981,
    1674.207031,
    1709.199951,
    1744.469971,
    1780.021973,
    1815.850952,
    1851.959961,
    1888.333984,
    1924.989014,
    1961.911011,
    1999.089966,
    2036.546021,
    2074.261963,
    2112.233887,
    2150.462891,
    2188.954102,
    2227.699951,
    2266.697021,
    2305.940918,
    2345.435059,
    2385.180908,
    2425.159912,
    2465.388916,
    2505.846924,
    2546.559082,
    2587.493896,
    2628.670898,
    2670.074951,
    2711.705078,
    2753.579102,
    2795.662109,
    2837.971924,
    2880.51001,
    2923.27002,
    2966.239014,
    3009.429932,
    3052.831055,
    3096.447021,
    3140.263916,
    3184.310059,
    3228.552979,
    3272.98999,
    3317.649902,
    3362.5,
    3407.548096,
    3452.795898,
    3498.24292,
    3543.878906,
    3589.702881,
    3635.727051,
    3681.937988,
    3728.331055,
    3774.907959,
    3821.674072,
    3868.616943,
    3915.731934,
    3963.039062,
    4010.506104,
    4058.163086,
    4105.987793,
    4153.984863,
    4202.146973,
    4250.487793,
    4298.966797,
    4347.626953,
    4396.454102,
    4445.434082,
    4494.575195,
    4543.878906,
    4593.331055,
    4642.938965,
    4692.696777,
    4742.609863,
    4792.666016,
    4842.868164,
    4893.221191,
    4943.708984,
    4994.356934,
    5045.134766,
    5096.048828,
    5147.094238,
    5198.279785,
    5223.944824,
    5249.606934,
    5301.062988,
    5352.658203,
    5404.36084,
    5456.212891,
    5508.169922,
    5560.282227,
    5612.487793,
    5664.821777,
    5717.266113,
    5769.845215,
    5822.554199,
    5875.347168,
    5928.262207,
    5981.300781,
    6034.458008,
    6087.696777,
    6141.054199,
    6194.508789,
    6248.080078,
    6274.911133,
    6301.745117,
    6355.512207,
    6409.395996,
    6463.367188,
    6517.424805,
    6571.588867,
    6598.713867,
    6625.838867,
    6680.192871,
    6734.625977,
    6789.143066,
    6816.444824,
    6871.108887,
    6925.845215,
    6953.23584,
    7008.10791,
    7063.030762,
    7090.568848,
    7118.070801,
    7173.173828,
    7200.762207,
    7256,
    7283.61377,
    7338.937988,
    7366.652832,
    7394.353027,
    7449.835938,
    7505.373047,
    7560.977051,
    7588.811035,
    7616.634766,
    7672.399902,
    7728.21582,
    7784.083008,
    7840.027832,
    7896.005859,
    7924.032227,
    7952.045898,
    8008.150879,
    8036.243164,
    8064.330078,
    8120.519043,
    8148.647949,
    8176.780762,
    8204.944336,
    8233.103516,
    8289.466797,
    8345.873047,
    8374.139648,
    8402.34082,
    8458.867188,
    8515.391602,
    8543.688477,
    8571.991211,
    8600.298828,
    8656.950195,
    8713.626953,
    8741.986328,
    8798.723633,
    8827.123047,
    8855.493164,
    8883.899414,
    8912.291992,
    8940.71875,
    8969.149414,
    9026.033203,
    9054.472656,
    9082.914062,
    9111.380859,
    9139.84668,
    9168.333984,
    9196.817383,
    9253.785156,
]

# generate_ballistics_table(375, 0, reforger_tools_drops)

In [169]:
def deg2EL(degrees):
    return math.radians(degrees) * 1000 - 960

In [ ]:
mortar_easting = 3273
mortar_northing = 2495
obs_easting = 2487
obs_northing = 3019
obs_azimuth = 322
obs_distance = 366
target_height = -32

res = azimuth_from_observer_relative(
    mortar_easting,
    mortar_northing,
    obs_easting,
    obs_northing,
    obs_azimuth,
    obs_distance,
)
print(res)
print(deg2EL(calculate_elevation(375, target_height, res[1], True)["elevation"]))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

for angle in range(0, 45):
    plot_points = ballistic_sim3(375, angle, 0, 0.001, True)[2]
    x = [point[0] for point in plot_points]
    y = [point[1] for point in plot_points]
    x = x[::10]
    y = y[::10]
    myplot = plt.plot(x, y)
    ax = plt.gca()
    ax.set_xlim(0, 5000)
    # test = ax.plot()[0]
    # test.set_data()
    plt.show()